In [42]:
from kaggle_secrets import UserSecretsClient
import google.generativeai as genai
import requests
from bs4 import BeautifulSoup
import os
from fpdf import FPDF
import shutil
from PIL import Image
import pytesseract
from sentence_transformers import SentenceTransformer
import numpy as np
import fitz  
import folium
from IPython.display import display
from geopy.distance import geodesic
import ee
import geemap

In [ ]:
#pip install fpdf


In [10]:
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("API_KEY")
secret_value_1 = user_secrets.get_secret("GEMINI_API_KEY_0")

genai.configure(api_key=secret_value_0)

#models = genai.list_models()

#print("Gemini Models Available (safe to list):")
#for m in models:
#    print("-", m.name)


In [ ]:
# import os
# from openai import OpenAI

# user_secrets = UserSecretsClient()
# secret_value_1 = user_secrets.get_secret("API_TOKEN")

# client = OpenAI(
#     base_url="https://models.inference.ai.azure.com",
#     api_key=secret_value_1,
# )



In [11]:
model = genai.GenerativeModel(model_name="gemini-2.0-flash")

print("Gemini model loaded successfully:")
print("Model name:", model.model_name)


Gemini model loaded successfully:
Model name: models/gemini-2.0-flash


In [ ]:
#pip install fpdf

In [12]:
folder = "paititi_articles"
os.makedirs(folder, exist_ok=True)

urls = {
    "jesuit_report": "https://www.paititi.info/discovering-paititi/jesuits-report/",
    "machupicchu_blog": "https://blog.viajesmachupicchu.travel/en/peruvian-legends-paititi-the-lost-gold-city-in-the-amazon/"
}

# Function to clean special characters not supported by fpdf
def clean_text(text):
    replacements = {
        '\u201c': '"', '\u201d': '"',     # curly double quotes
        '\u2018': "'", '\u2019': "'",     # curly single quotes
        '\u2014': '-',  '\u2013': '-',    # em/en dashes
        '\u2026': '...', '\xa0': ' '      # ellipsis, non-breaking space
    }
    for src, target in replacements.items():
        text = text.replace(src, target)
    return text

def txt_to_pdf(txt_path, pdf_path):
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()
    pdf.set_font("Arial", size=12)

    with open(txt_path, "r", encoding="utf-8") as f:
        for line in f:
            clean_line = clean_text(line.strip())
            pdf.multi_cell(0, 10, clean_line)

    pdf.output(pdf_path)

def scrape_article(name, url):
    print(f"Scraping: {url}")
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    # Extract article text
    paragraphs = [p.get_text(strip=True) for p in soup.find_all("p")]
    full_text = "\n\n".join(paragraphs)

    # Save .txt
    text_path = os.path.join(folder, f"{name}_text.txt")
    with open(text_path, "w", encoding="utf-8") as f:
        f.write(full_text)

    # Convert to PDF
    pdf_path = os.path.join(folder, f"{name}_text.pdf")
    txt_to_pdf(text_path, pdf_path)

    # Download all images
    img_tags = soup.find_all("img")
    for i, img in enumerate(img_tags):
        img_url = img.get("src")
        if not img_url or img_url.startswith("data:"):
            continue

        full_img_url = requests.compat.urljoin(url, img_url)
        try:
            img_data = requests.get(full_img_url).content
            img_ext = os.path.splitext(full_img_url)[1].split("?")[0] or ".jpg"
            img_name = f"{name}_img_{i}{img_ext}"
            with open(os.path.join(folder, img_name), "wb") as f:
                f.write(img_data)
        except Exception as e:
            print(f" Failed to download image: {full_img_url} — {e}")

# Run for each URL
for name, url in urls.items():
    scrape_article(name, url)

print("\nScraping complete! Text, images, and PDFs saved in the 'paititi_articles' folder.")


Scraping: https://www.paititi.info/discovering-paititi/jesuits-report/
Scraping: https://blog.viajesmachupicchu.travel/en/peruvian-legends-paititi-the-lost-gold-city-in-the-amazon/

Scraping complete! Text, images, and PDFs saved in the 'paititi_articles' folder.


In [13]:
article_folder = "/kaggle/working/paititi_articles"
pdf_paths = [
    "/kaggle/input/library-of-congress/Library of Congress.pdf",
    "/kaggle/input/archsitesintheperuvianamazonusingsatellite/IdentificationofArcheologicalSitesinthePeruvianAmazonUsingSatellite.pdf"
]


os.makedirs(article_folder, exist_ok=True)

# Move the two PDF files into the `paititi_articles` folder
for pdf_path in pdf_paths:
    filename = os.path.basename(pdf_path)
    dest_path = os.path.join(article_folder, filename)
    shutil.copy(pdf_path, dest_path)

print("PDFs moved to:", article_folder)


PDFs moved to: /kaggle/working/paititi_articles


In [14]:
files_to_delete = [
    "/kaggle/working/paititi_articles/machupicchu_blog_img_1.png",
    "/kaggle/working/paititi_articles/machupicchu_blog_img_11.jpg",
    "/kaggle/working/paititi_articles/machupicchu_blog_img_13.jpg",
    "/kaggle/working/paititi_articles/machupicchu_blog_img_3.webp",
    "/kaggle/working/paititi_articles/machupicchu_blog_img_5.jpg",
    "/kaggle/working/paititi_articles/machupicchu_blog_img_7.jpg",
    "/kaggle/working/paititi_articles/machupicchu_blog_img_9.jpg",
    "/kaggle/working/paititi_articles/jesuit_report_text.txt",
    "/kaggle/working/paititi_articles/jesuit_report_img_25.png",
    "/kaggle/working/paititi_articles/jesuit_report_img_4.png",
    "/kaggle/working/paititi_articles/jesuit_report_img_5.png",
    "/kaggle/working/paititi_articles/jesuit_report_img_7.jpg",
    "/kaggle/working/paititi_articles/machupicchu_blog_text.txt"
]

for filename in files_to_delete:
    file_path = os.path.join(folder, filename)
    if os.path.exists(file_path):
        os.remove(file_path)
        print(f"Deleted: {filename}")
    else:
        print(f" Not found: {filename}")



Deleted: /kaggle/working/paititi_articles/machupicchu_blog_img_1.png
Deleted: /kaggle/working/paititi_articles/machupicchu_blog_img_11.jpg
Deleted: /kaggle/working/paititi_articles/machupicchu_blog_img_13.jpg
Deleted: /kaggle/working/paititi_articles/machupicchu_blog_img_3.webp
Deleted: /kaggle/working/paititi_articles/machupicchu_blog_img_5.jpg
Deleted: /kaggle/working/paititi_articles/machupicchu_blog_img_7.jpg
Deleted: /kaggle/working/paititi_articles/machupicchu_blog_img_9.jpg
Deleted: /kaggle/working/paititi_articles/jesuit_report_text.txt
Deleted: /kaggle/working/paititi_articles/jesuit_report_img_25.png
Deleted: /kaggle/working/paititi_articles/jesuit_report_img_4.png
Deleted: /kaggle/working/paititi_articles/jesuit_report_img_5.png
Deleted: /kaggle/working/paititi_articles/jesuit_report_img_7.jpg
Deleted: /kaggle/working/paititi_articles/machupicchu_blog_text.txt


### Prompting

In [15]:
model_sentence = SentenceTransformer("all-MiniLM-L6-v2")

folder_path = "/kaggle/working/paititi_articles"
file_embeddings = {}

def extract_text(file_path):
    text = ""
    with fitz.open(file_path) as doc:
        for page in doc:
            text += page.get_text()
    return text

# Extract and embed only PDFs
for file_name in os.listdir(folder_path):
    if file_name.lower().endswith(".pdf"):
        path = os.path.join(folder_path, file_name)
        text = extract_text(path)
        if text.strip():
            embedding = model_sentence.encode(text)
            file_embeddings[file_name] = {
                "path": path,
                "embedding": embedding,
                "text": text
            }

print(f" Embedded {len(file_embeddings)} PDF files.")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

 Embedded 4 PDF files.


In [20]:
def get_top_k_files(user_query, k=3):
    query_embedding = model_sentence .encode(user_query)
    similarity_scores = []

    for file_name, data in file_embeddings.items():
        score = np.dot(query_embedding, data["embedding"])  # cosine similarity
        similarity_scores.append((file_name, score))

    # Sort by highest similarity score
    top_k = sorted(similarity_scores, key=lambda x: x[1], reverse=True)[:k]

    print(f"\n Top {k} files for query: \"{user_query}\"")
    for rank, (file_name, score) in enumerate(top_k, 1):
        print(f"{rank}. {file_name} (score: {score:.4f})")

    return [file_embeddings[name]["path"] for name, _ in top_k]


In [21]:
top_files = get_top_k_files("Files: ", k=3)
uploaded_files = [genai.upload_file(path) for path in top_files] 

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


 Top 3 files for query: "Files: "
1. IdentificationofArcheologicalSitesinthePeruvianAmazonUsingSatellite.pdf (score: 0.0822)
2. jesuit_report_text.pdf (score: 0.0455)
3. Library of Congress.pdf (score: 0.0412)


In [22]:
query = """
You are an expert historian and archaeologist. Carefully read the attached historical documents related to the legend of Paititi, including colonial-era Jesuit reports, indigenous testimonies, and modern expedition books.

Your task is to extract and analyze all possible clues, mentions, or indirect references to:

- The city of Paititi or any related Inca settlements, sacred cities, or mythical regions
- Named towns, villages, rivers, lakes, mountains, or other natural landmarks near or on the path to Paititi
- Descriptions of journeys, routes, or paths taken by Jesuit missionaries, indigenous guides, or explorers (especially east, northeast, or southeast of Cusco)
- Terrain references such as jungle, cliffs, valleys, fog forests, swamps, or unusual ecosystems
- Cultural or ritualistic references related to gold, treasure, Inca exile, guardians, or sacred protections

For **each meaningful clue**, give the following in structured format:

1. **Quote or Paraphrased Text** – Pull a direct excerpt or rephrase clearly
2. **Interpretation** – What does it likely mean historically or geographically?
3. **Modern Region** – Suggest a plausible modern region this refers to
4. **Estimated Coordinates:**
   - Latitude: [decimal]
   - Longitude: [decimal]
5. **Confidence Level** – High / Medium / Low (based on clarity of reference)
6. **Source Document** – Name of the book, report, or manuscript
7. **Historical Period** – If identifiable (e.g., 1600s Jesuit, 1900s explorer)

If multiple clues describe the **same or adjacent locations**, group them as a **Clue Cluster** and describe the relationship.

---

Finally, give a **Search Hypothesis Summary**:
- Where should modern researchers focus exploration?
- Suggest 1–3 **Target Zones** with:
   - A brief explanation for each
   - Coordinates (Latitude and Longitude separately)
- Highlight any **clustering patterns** around specific rivers, forests, ruins, or mapped anomalies

Format your answer to be **scholarly, precise, and GIS-compatible** — so it can be used directly in Google Earth Engine, GIS tools, or expedition planning.
"""


In [23]:
combined_text = ""
for file_data in file_embeddings.values():
    combined_text += f"\n\n--- File: {os.path.basename(file_data['path'])} ---\n\n"
    combined_text += file_data["text"]

full_prompt = combined_text + "\n\n" + query
response = model.generate_content(full_prompt)


In [24]:
print(response.text)


Okay, I have carefully reviewed the provided documents related to the legend of Paititi and will extract and analyze the relevant clues according to your specified format.

**Clue 1**
1.  **Quote or Paraphrased Text**: "A legendary city of gold... in the east of the Andes, deep within the Peruvian Amazon rainforest... north of Bolivia and southwest of Brazil, connecting it to the famous 'El Dorado'."
2.  **Interpretation**: This is the general geographical description of Paititi. Emphasis on the *eastern Andes* and the connection to *El Dorado* suggests focusing on the transition zone between the Andes and the Amazon basin. The link implies a potential existence of gold resources.
3.  **Modern Region**: Southeastern Peru, possibly extending into parts of Bolivia and Brazil.
4.  **Estimated Coordinates:**
    *   Latitude: -12.00 (approximate center of the region)
    *   Longitude: -70.00 (approximate center of the region)
5.  **Confidence Level**: Low (very general description)
6.  **

## Image Prompting

In [25]:
manuscript_images = [
    "/kaggle/working/paititi_articles/jesuit_report_img_11.jpg",
    "/kaggle/working/paititi_articles/jesuit_report_img_13.jpg",
    "/kaggle/working/paititi_articles/jesuit_report_img_15.jpg",
    "/kaggle/working/paititi_articles/jesuit_report_img_17.jpg",
    "/kaggle/working/paititi_articles/jesuit_report_img_19.jpg",
    "/kaggle/working/paititi_articles/jesuit_report_img_9.jpg"
]

manuscript_prompt = """
You are a historical linguist and archaeologist with expertise in ancient and colonial documents related to the Inca civilization.

You have been given an image of a handwritten manuscript that may relate to the legend of Paititi — the lost city of gold. Do not transcribe the document.

Instead, please analyze the image holistically and provide insights based on:

1. Stylistic features — What kind of document might this be? A report? A diary? A letter? Does the writing style suggest it’s colonial, missionary, or indigenous in origin?
2. Mentioned names or terms — Can you visually recognize recurring words, especially proper nouns like towns, regions, or rivers?
3. Language or script clues — Does the writing appear Spanish, Quechua, Latin, or a mix? Is the tone formal, poetic, or instructional?
4. Directional or geographic hints — Even if unclear, are there parts that resemble geographic terms or journey descriptions (e.g., directions, distances, places)?
5. Possible connections to Paititi — Based on visible content, could this page relate to Inca exile, sacred gold, lost settlements, or missionary paths into the Amazon?
6. Provide a reasoned hypothesis: If this document is relevant, what does it imply about the location or myth of Paititi?

Your goal is to help archaeologists understand what this manuscript might reveal about the path to Paititi — or how it was imagined by the writer.

Please give your findings as a professional interpretation, not as a transcription.
"""

for img_path in manuscript_images:
    print(f"\n Analyzing: {os.path.basename(img_path)}")
    image = Image.open(img_path)

    response = model.generate_content([image, manuscript_prompt])
    print(response.text)


 Analyzing: jesuit_report_img_11.jpg
Okay, based on the image and the additional crops provided, here's my interpretation of the manuscript page, considering its potential relevance to the legend of Paititi:

**Overall Impression:**

The document appears to be a narrative account, possibly a report or a personal journal/diary entry. The handwriting is cursive and seems consistent throughout the page, indicating a single author. The overall tone, based on the glimpses I can discern, seems descriptive and observational, rather than formally legal or strictly academic. The writing style suggests a colonial origin.

**Stylistic Features:**

*   The fluid handwriting, with frequent ligatures and abbreviations, indicates a trained scribe or someone accustomed to writing extensively.
*   The page layout lacks strict formatting, implying this wasn't a formal, officially-published document.
*   The style is not indigenous in origin, as indigenous people would not use the Latin alphabet and cur

In [27]:
images = [
    Image.open("/kaggle/working/paititi_articles/jesuit_report_img_11.jpg"),
    Image.open("/kaggle/working/paititi_articles/jesuit_report_img_13.jpg"),
    Image.open("/kaggle/working/paititi_articles/jesuit_report_img_15.jpg"),
    Image.open("/kaggle/working/paititi_articles/jesuit_report_img_17.jpg"),
    Image.open("/kaggle/working/paititi_articles/jesuit_report_img_19.jpg"),
    Image.open("/kaggle/working/paititi_articles/jesuit_report_img_9.jpg")
]

response = model.generate_content([
    "Please transcribe all the following manuscript images together then translate it in English and identify possible connections to the legend of Paititi. Focus on terms like 'Regno', 'Indiani', 'oro', 'Crocifisso', and any references to sacred places or missionary journeys.",
    *images
])

print(response.text)

Here's a transcription of the manuscript images, followed by an English translation and an analysis of potential connections to the Paititi legend.  Note that due to the cursive script and some unclear portions, there might be minor inaccuracies in the transcription.

**Transcription (All Images Combined):**

**(Image 1)**

che cosa sono quel Crocifisso, come aspetti di veder la formal un huomo messo in un legno, risposero li detti Indiani; ch'era la figura del Dio de' Christiani. Alli hora lo guardiarono no vollessoro, videndosene con ogni meno prezzo, e lasciandolo passare pure liberam.te nel Regno, arrivorono nella Città dove tenena la sua Corte il Re del detto Regno di Paititi; et avendo il Re avviso dell' arrivata delli detti Indiani, et che portavano seco solo la Figura del Dio, che Christiani adorano per curiosità [incomprehensible] fecero chiamare presenza sua, et di tutta la sua Corte, dove concorsero tutti li principali Signori della sua Corte; per curiosità vanno per vedere 

## Transcription & Analysis of Manuscript Images Related to the Paititi Legend

#### **Image 1** – Encounter with the King of Paititi

> "...What is that crucifix? Upon seeing it—a figure embedded in wood—the Indians responded that it was the image of the Christian God. At first, the King and his court laughed and mocked the crucifix. But then, something miraculous occurred: the crucifix lifted its head and moved its eyes. Witnessing this, the King became afraid and declared, "Great is the God of the Christians!" and proceeded to worship the crucifix."

* **Keywords:** Kingdom of Paititi, King, Crucifix, Miracle, Indigenous response
* **Key Clue:** The "Regno di Paytiti" is explicitly mentioned.

---

#### **Image 2** – Religious Conversion and Gold Chapel

> "...The King, now convinced of the Christian God's power, built a chapel in his palace courtyard made entirely of fine gold, adorned with gems. He worshipped the crucifix daily, along with his court and son. The King later requested missionaries to come and teach the Christian law."

* **Keywords:** Gold chapel, Christian law, daily worship, conversion
* **Key Clue:** A chapel made of **pure gold** is a direct allusion to Paititi's legendary wealth.

---

#### **Image 3** – Mission to the Kingdom of Pajaziti

> "...Father Andrea Lopez baptized the King and his son. Before dying of a fever, the King asked that the Christian faith be maintained in his kingdom. The King promised to build a church with solid gold walls and requested Father Andrea to lead a mission to Pajaziti."

* **Keywords:** Pajaziti Kingdom, gold walls, church, Father Andrea Lopez
* **Key Clue:** **Pajaziti** is a variant spelling of Paititi; this document explicitly calls for a mission to the kingdom.

---

#### **Image 4** – Worship of a Sacred Healing Stone

> "...The Indians worshipped a large, bizarre stone ('pietra bazzarra'), which they believed cured diseases through miraculous powers. Andrea Lopez told them this was deception from the Devil and that they should worship the God who gave the stone its power."

* **Keywords:** Sacred stone, miracles, healing, indigenous beliefs, conversion
* **Key Clue:** Suggests indigenous worship of sacred elements, potentially tying to rituals in Paititi.

---

#### **Image 5** – *(Too blurry to transcribe)*

---

#### **Image 6** – Description of Tajaiti (Paititi)

> "...From the Kingdom of Tajaiti... full of miracles... Father Andrea Lopez, rector of the Jesuit college... described the Indians, their conversion, and their riches of gold and silver. The region had impressive customs, people with gold and silver accessories, and nobility."

* **Keywords:** Tajaiti Kingdom, riches, conversion, Father Lopez, gold/silver, nobility
* **Key Clue:** Tajaiti is possibly another variant of Paititi. The description matches the mythical wealth and structure of the city.

---

### 🔍 Overall Analysis and Paititi Connections

**1. Explicit Mentions of Paititi**

* Names: Paytiti, Pajaziti, Tajaiti (all variants)
* These variants reinforce the manuscript's direct reference to the legendary kingdom.

**2. Description of Wealth**

* Mentions of: solid gold chapels, golden palace courtyards, churches made of "massive gold"
* Strongly supports the legendary status of Paititi as a city of immense riches.

**3. Christian Influence and Missionary Activity**

* The main figure: **Father Andrea Lopez**
* Documents describe religious conversion of the King and people of Paititi

**4. Sacred Indigenous Beliefs**

* Worship of a healing stone suggests cultural and spiritual complexity, possibly indicating the city's sacred geography.

**5. Geographic Hints and Frontier Language**

* Mentions of leagues of travel, unknown frontiers, and gold-rich regions
* Could aid in mapping approximate areas for archaeological interest

---

### 📚 Research Directions

1. **Historical Validation**: Confirm the identity and activities of Father Andrea Lopez via Jesuit archives.
2. **Linguistic Clues**: Study terms like "pietra bazzarra" and place names like "Ruges" for possible indigenous etymology.
3. **Geographic Mapping**: Use distance measures and frontier descriptions to align with modern Amazon/Andes regions.
4. **Archaeological Fieldwork**: Investigate gold artifacts or sacred stone practices tied to healing in the Upper Amazon.

---

### 📌 Final Hypothesis

These manuscripts provide **highly compelling circumstantial evidence** for the historical basis of the Paititi legend. With repeated references to Christian missions, vast gold resources, a structured indigenous kingdom, and spiritually significant elements, these documents should be considered vital leads in the academic and archaeological pursuit of the lost city of Paititi.


>  The manuscript presents a Jesuit-era account of the Kingdom of Paytiti, also referred to as Pajaziti or Tajaiti. It describes how Indigenous messengers brought a crucifix to the King of this kingdom. At first, the King and his court mocked the Christian symbol. However, a miraculous event reportedly occurred when the crucifix raised its head and appeared to move its eyes. Witnessing this, the King became convinced of the Christian God’s power. He converted to Christianity, built a chapel made of fine gold to house the crucifix, and began daily worship with his court.

> Following his conversion, the King requested Jesuit missionaries to visit the kingdom to teach Christian doctrine. He promised to build a college and a church made of solid gold. The account identifies a missionary named Andrea López, who baptized the King and his court. It also describes the Indigenous people’s earlier worship of a sacred healing stone, which they believed cured illness through divine power. The Jesuits rejected this belief and claimed it was a deception of the devil.

> Throughout the document, there are several references to gold, silver, and precious gems, along with mentions of a well-structured kingdom ruled by a monarch. These details support the idea that Paytiti may have been a real place—an organized, wealthy Indigenous state with strong spiritual traditions, later influenced by Christian missionaries during colonial exploration.

### Mapping

In [28]:
m = folium.Map(location=[-12.8, -71.4], zoom_start=7)

locations = [
    {"name": "Nistron River, Manu National Park", "lat": -12.850, "lon": -71.450},
    {"name": "Pantiacolla Jungle (Paititi suspect zone)", "lat": -13.000, "lon": -71.500},
    
    {"name": " City of Paititi (as referenced in manuscript)", "lat": -12.500, "lon": -72.000},
    {"name": " Location of Crucifix Miracle (as described by Indian 'Chausea')", "lat": -12.600, "lon": -71.700},
    {"name": " Site where gold chapel was allegedly built", "lat": -12.520, "lon": -71.950},
    {"name": " Missionary Route of P. Andrea Lopez", "lat": -12.000, "lon": -76.900},  # Lima region, origin point
    {"name": " College of the Company of Jesus (Cusco)", "lat": -13.516, "lon": -71.978},
    {"name": " Corsen Valley, Peru (possibly Cotahuasi area)", "lat": -15.200, "lon": -72.650},
    {"name": " Sacred Stone (Pietra Bazzarra) Healing Site", "lat": -12.700, "lon": -72.100},
    {"name": " Alleged Transfer Route to the Pope (from Paititi)", "lat": 41.9028, "lon": 12.4964},  # Vatican City, symbolic
]

for loc in locations:
    folium.Marker(
        location=[loc["lat"], loc["lon"]],
        popup=folium.Popup(loc["name"], max_width=300),
        icon=folium.Icon(color="green", icon="leaf")
    ).add_to(m)

display(m)


In [31]:
m = folium.Map(
    location=[-12.8, -71.4],
    zoom_start=7,
    tiles="OpenStreetMap",
    attr="© OpenStreetMap contributors"
)

locations = [
    {
        "name": "AOI3: Nistron River (Satellite AOI)",
        "lat": -12.850,
        "lon": -71.450,
        "color": "red",
        "radius": 10000,
        "popup": """
        <b>Nistron River, Manu National Park</b><br>
        Identified via satellite remote sensing (AOI3) as having ideal terrain and possible ancient structures.
        """
    },
    {
        "name": "Pantiacolla Jungle (Inkarri Refuge)",
        "lat": -13.000,
        "lon": -71.500,
        "color": "green",
        "radius": 8000,
        "popup": """
        <b>Pantiacolla Region</b><br>
        Said to be where Inkarri found peace. Deep in jungle, linked to Q'ero people and Inca legends.
        """
    },
    {
        "name": "Madre de Dios (Fawcett's Zone)",
        "lat": -12.500,
        "lon": -70.000,
        "color": "blue",
        "radius": 12000,
        "popup": """
        <b>Madre de Dios Region</b><br>
        Percy Fawcett's search area; satellite reveals geometric formations suggesting ancient settlements.
        """
    },
    {
        "name": "Pongo de Mainique (Indigenous Route)",
        "lat": -12.140,
        "lon": -73.400,
        "color": "orange",
        "radius": 7000,
        "popup": """
        <b>Pongo de Mainique</b><br>
        Machiguenga stories speak of ancient gold being hidden past this gorge along the Urubamba River.
        """
    },
    {
        "name": "City of Paititi (as referenced in manuscript)",
        "lat": -12.500,
        "lon": -72.000,
        "color": "purple",
        "radius": 10000,
        "popup": """
        <b>City of Paititi (Historical Reference)</b><br>
        Described in Jesuit manuscript as the capital visited by missionaries and site of conversion.
        """
    },
    {
        "name": "Location of Crucifix Miracle (Indian 'Chausea')",
        "lat": -12.600,
        "lon": -71.700,
        "color": "darkred",
        "radius": 5000,
        "popup": """
        <b>Miracle Site</b><br>
        Site where the crucifix allegedly moved and caused the king’s conversion.
        """
    },
    {
        "name": "Site of Golden Chapel (Built by King)",
        "lat": -12.520,
        "lon": -71.950,
        "color": "gold",
        "radius": 6000,
        "popup": """
        <b>Golden Oratory Site</b><br>
        Location where the King built a chapel entirely adorned in gold and gems.
        """
    },
    {
        "name": "Sacred Stone (Pietra Bazzarra) Healing Site",
        "lat": -12.700,
        "lon": -72.100,
        "color": "darkgreen",
        "radius": 5000,
        "popup": """
        <b>Sacred Stone Worship Site</b><br>
        Healing stone worshipped before conversion; later offered to the Pope.
        """
    },
    {
        "name": "Missionary Route Start (Lima)",
        "lat": -12.0464,
        "lon": -77.0428,
        "color": "cadetblue",
        "radius": 3000,
        "popup": """
        <b>Missionary Origin Point</b><br>
        Likely origin of Jesuit missionary P. Andrea Lopez’s journey into the Amazon.
        """
    },
    {
        "name": "Jesuit College in Cusco",
        "lat": -13.516,
        "lon": -71.978,
        "color": "lightblue",
        "radius": 3000,
        "popup": """
        <b>College of the Company of Jesus</b><br>
        Jesuit headquarters; possibly coordinated missionary efforts to Paititi.
        """
    },
    {
        "name": "Corsen Valley (Possibly Cotahuasi)",
        "lat": -15.200,
        "lon": -72.650,
        "color": "gray",
        "radius": 4000,
        "popup": """
        <b>Corsen Valley</b><br>
        Mentioned in manuscript as Andrea Lopez’s base; likely Cotahuasi region.
        """
    },
    {
        "name": "Alleged Transfer to the Pope (Symbolic)",
        "lat": 41.9028,
        "lon": 12.4964,
        "color": "pink",
        "radius": 20000,
        "popup": """
        <b>Vatican City</b><br>
        The sacred healing stone was reportedly sent here for preservation.
        """
    },
]

for loc in locations:
    folium.Marker(
        [loc["lat"], loc["lon"]],
        popup=folium.Popup(loc["popup"], max_width=300),
        icon=folium.Icon(color=loc["color"], icon="info-sign")
    ).add_to(m)
    folium.Circle(
        radius=loc["radius"],
        location=[loc["lat"], loc["lon"]],
        color=loc["color"],
        fill=True,
        fill_opacity=0.3
    ).add_to(m)

coords = {loc["name"]: (loc["lat"], loc["lon"]) for loc in locations}

route_sequence = [
    "Missionary Route Start (Lima)",
    "Jesuit College in Cusco",
    "Corsen Valley (Possibly Cotahuasi)",
    "Pantiacolla Jungle (Inkarri Refuge)",
    "AOI3: Nistron River (Satellite AOI)",
    "Location of Crucifix Miracle (Indian 'Chausea')",
    "Site of Golden Chapel (Built by King)",
    "City of Paititi (as referenced in manuscript)",
    "Sacred Stone (Pietra Bazzarra) Healing Site",
    "Madre de Dios (Fawcett's Zone)",
    "Alleged Transfer to the Pope (Symbolic)"
]

for i in range(len(route_sequence) - 1):
    start_name = route_sequence[i]
    end_name = route_sequence[i + 1]
    start_coords = coords[start_name]
    end_coords = coords[end_name]
    dist_km = geodesic(start_coords, end_coords).km

    folium.PolyLine(
        [start_coords, end_coords],
        color="purple",
        weight=2.5,
        opacity=0.7,
        tooltip=f"{start_name} → {end_name}: {dist_km:.1f} km"
    ).add_to(m)

display(m)


/tmp/ipykernel_35/3089992488.py:153: UserWarning: color argument of Icon should be one of: {'beige', 'lightgray', 'lightgreen', 'pink', 'purple', 'orange', 'red', 'green', 'darkpurple', 'darkred', 'blue', 'darkblue', 'lightblue', 'darkgreen', 'cadetblue', 'lightred', 'gray', 'white', 'black'}.
  icon=folium.Icon(color=loc["color"], icon="info-sign")


In [ ]:
m = folium.Map(
    location=[-12.8, -71.4],
    zoom_start=8,
     tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
    attr="Tiles © Esri & the GIS community" )
for loc in locations:
    folium.Marker(
        [loc["lat"], loc["lon"]],
        popup=folium.Popup(loc["popup"], max_width=300),
        icon=folium.Icon(color=loc["color"].lower(), icon="info-sign")
    ).add_to(m)

    folium.Circle(
        radius=loc["radius"],
        location=[loc["lat"], loc["lon"]],
        color=loc["color"],
        fill=True,
        fill_opacity=0.3
    ).add_to(m)

# Draw path (explorer route estimate between major clues)
explorer_route = [
    [-13.000, -71.500],  # Pantiacolla
    [-12.850, -71.450],  # Nistron AOI
    [-12.500, -70.000]   # Madre de Dios
]
folium.PolyLine(explorer_route, color="purple", weight=3, opacity=0.7).add_to(m)


# Show the map
display(m)


In [33]:
m = folium.Map(
    location=[-12.8, -71.4],
    zoom_start=7,
    tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
attr="Tiles © Esri & the GIS community"
)

for loc in locations:
    folium.Marker(
        [loc["lat"], loc["lon"]],
        popup=folium.Popup(loc["popup"], max_width=300),
        icon=folium.Icon(color=loc["color"], icon="info-sign")
    ).add_to(m)
    folium.Circle(
        radius=loc["radius"],
        location=[loc["lat"], loc["lon"]],
        color=loc["color"],
        fill=True,
        fill_opacity=0.3
    ).add_to(m)

coords = {loc["name"]: (loc["lat"], loc["lon"]) for loc in locations}

route_sequence = [
    "Missionary Route Start (Lima)",
    "Jesuit College in Cusco",
    "Corsen Valley (Possibly Cotahuasi)",
    "Pantiacolla Jungle (Inkarri Refuge)",
    "AOI3: Nistron River (Satellite AOI)",
    "Location of Crucifix Miracle (Indian 'Chausea')",
    "Site of Golden Chapel (Built by King)",
    "City of Paititi (as referenced in manuscript)",
    "Sacred Stone (Pietra Bazzarra) Healing Site",
    "Madre de Dios (Fawcett's Zone)",
    "Alleged Transfer to the Pope (Symbolic)"
]

for i in range(len(route_sequence) - 1):
    start_name = route_sequence[i]
    end_name = route_sequence[i + 1]
    start_coords = coords[start_name]
    end_coords = coords[end_name]
    dist_km = geodesic(start_coords, end_coords).km

    folium.PolyLine(
        [start_coords, end_coords],
        color="purple",
        weight=2.5,
        opacity=0.7,
        tooltip=f"{start_name} → {end_name}: {dist_km:.1f} km"
    ).add_to(m)

display(m)


/tmp/ipykernel_35/576161357.py:12: UserWarning: color argument of Icon should be one of: {'beige', 'lightgray', 'lightgreen', 'pink', 'purple', 'orange', 'red', 'green', 'darkpurple', 'darkred', 'blue', 'darkblue', 'lightblue', 'darkgreen', 'cadetblue', 'lightred', 'gray', 'white', 'black'}.
  icon=folium.Icon(color=loc["color"], icon="info-sign")


## Google Earth Engine

In [45]:
#!pip install earthengine-api geemap
#!earthengine unauthenticate


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


E0000 00:00:1750884409.995315     221 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750884410.002021     221 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
usage: earthengine [-h] [--ee_config EE_CONFIG] [--service_account_file SERVICE_ACCOUNT_FILE]
                   [--project PROJECT_OVERRIDE] [--version]
                   {authenticate,acl,asset,cp,create,ls,alpha,du,mv,model,rm,set_project,task,unset_project,upload,upload_manifest,upload_table_manifest}
                   ...
earthengine: error: argument main_cmd: invalid choice: 'unauthenticate' (choose from 'authenticate', 'acl', 'asset', 'cp', 'create', 'ls', 'alpha', 'du', 'mv', 'model', 'rm', 'set_project', 'task', 'unset_project', 'upload', 'upload_manifest', 'upload_table_manifest')


In [56]:
ee.Authenticate(auth_mode='notebook')

ee.Initialize(project='paititi-464021')

print("Earth Engine initialized with project:", 'paititi-464021')


Earth Engine initialized with project: paititi-464021


In [57]:
Map = geemap.Map(center=[-12.8, -71.4], zoom=8)

# Sentinel-2 imagery (median composite of 2023)
sentinel = ee.ImageCollection("COPERNICUS/S2_SR") \
    .filterBounds(ee.Geometry.Point([-71.4, -12.8])) \
    .filterDate('2023-01-01', '2023-12-31') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
    .median()

true_color = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 3000,
    'gamma': 1.2
}
Map.addLayer(sentinel, true_color, "Sentinel-2 True Color")

# Add Elevation
elevation = ee.Image("USGS/SRTMGL1_003")
Map.addLayer(elevation, {"min": 0, "max": 3000}, "Elevation")

# Add NDVI
ndvi = sentinel.normalizedDifference(['B8', 'B4'])
Map.addLayer(ndvi, {"min": 0, "max": 1, "palette": ['white', 'green']}, "NDVI")

locations = [
    ["AOI3: Nistron River", -12.850, -71.450],
    ["Pantiacolla", -13.000, -71.500],
    ["Madre de Dios", -12.500, -70.000],
    ["Pongo de Mainique", -12.140, -73.400],
    ["City of Paititi", -12.500, -72.000],
    ["Crucifix Miracle", -12.600, -71.700],
    ["Golden Chapel", -12.520, -71.950],
    ["Sacred Stone", -12.700, -72.100],
    ["Mission Start (Lima)", -12.0464, -77.0428],
    ["Jesuit College", -13.516, -71.978],
    ["Corsen Valley", -15.200, -72.650],
]

for name, lat, lon in locations:
    Map.add_marker(location=[lat, lon], popup=name)

Map

TimeoutException: Requesting secret GOOGLE_MAPS_API_KEY timed out. Secrets can only be fetched when running from the Colab UI.

In [59]:
import geemap


In [62]:
import ee
import geemap

# Initialize Earth Engine (make sure you've authenticated and initialized previously)
ee.Initialize()

# Create the map centered on your region
Map = geemap.Map(center=[-12.8, -71.4], zoom=8)

# Load Sentinel-2 imagery (median composite 2023)
sentinel = (ee.ImageCollection("COPERNICUS/S2_SR")
            .filterBounds(ee.Geometry.Point([-71.4, -12.8]))
            .filterDate('2023-01-01', '2023-12-31')
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
            .median())

true_color = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 3000,
    'gamma': 1.2
}

Map.addLayer(sentinel, true_color, "Sentinel-2 True Color")

# Add Elevation layer
elevation = ee.Image("USGS/SRTMGL1_003")
Map.addLayer(elevation, {"min": 0, "max": 3000}, "Elevation")

# Add NDVI layer
ndvi = sentinel.normalizedDifference(['B8', 'B4'])
Map.addLayer(ndvi, {"min": 0, "max": 1, "palette": ['white', 'green']}, "NDVI")

# Locations to mark
locations = [
    ["AOI3: Nistron River", -12.850, -71.450],
    ["Pantiacolla", -13.000, -71.500],
    ["Madre de Dios", -12.500, -70.000],
    ["Pongo de Mainique", -12.140, -73.400],
    ["City of Paititi", -12.500, -72.000],
    ["Crucifix Miracle", -12.600, -71.700],
    ["Golden Chapel", -12.520, -71.950],
    ["Sacred Stone", -12.700, -72.100],
    ["Mission Start (Lima)", -12.0464, -77.0428],
    ["Jesuit College", -13.516, -71.978],
    ["Corsen Valley", -15.200, -72.650],
]

for name, lat, lon in locations:
    Map.add_marker(location=[lat, lon], popup=name)

Map


TimeoutException: Requesting secret GOOGLE_MAPS_API_KEY timed out. Secrets can only be fetched when running from the Colab UI.